In [ ]:
### THIS DIFFERS FROM OTHER BENCHMARK FILES BECAUSE IT WAS THE FIRST ATTEMPT AT BENCHMARKING
### RE-RUNS WITH IMPROVED CODE WERE NOT DONE SINCE THEY ARE VERY TIME- AND RESOURCE-CONSUMING

import whisper_timestamped as whisper
import json
import os
import time
import torch
import datetime
import gc

start = time.time()
model = whisper.load_model('large-v3', device="cuda:0")

for file in os.listdir('/evaluation/jasmin_comp_p/5_sil/'):
    audio = whisper.load_audio('/evaluation/jasmin_comp_p/5_sil/' + file)
    result = whisper.transcribe(model, audio, vad=True, beam_size=5, best_of=5, temperature=(0.0, 0.2, 0.4, 0.6, 0.8, 1.0), language="nl")
    with open('/evaluation/jasmin_comp_p/5_res/vad3/' + file[:-3] + 'json', 'w', encoding='utf-8') as f:
        json.dump(result, f, indent = 2, ensure_ascii = False)
        print(file[:-4] + ' has been transcribed')
        print(f"gpu used {torch.cuda.max_memory_allocated(device='cuda:0') / float(1e9)} GB memory")
        torch.cuda.reset_peak_memory_stats(device='cuda:0')

end = time.time()
time_s = end - start
print('Time spent evaluating: ' + str(datetime.timedelta(seconds=time_s)))
with open('/evaluation/jasmin_comp_p/5_res/vad3/a_time_spent.txt', 'w') as f:
    f.write(str(datetime.timedelta(seconds=time_s)))

del model
torch.cuda.empty_cache()
gc.collect()